In [36]:
import os
import shutil
from urllib.parse import urlparse

import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [42]:
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1200,600')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = 'https://search.shopping.naver.com/search/all?adQuery=%EC%82%AC%EB%A3%8C&frm=NVSHATC&origQuery=%EC%82%AC%EB%A3%8C&pagingIndex=1&pagingSize=40&productSet=checkout&query=%EC%82%AC%EB%A3%8C&sort=rel&timestamp=&viewType=image'

driver.get(url)
driver.implicitly_wait(5)
driver.execute_script('window.scrollBy(0, document.body.scrollHeight)')
driver.implicitly_wait(5)

product_pages = driver.find_elements(By.CSS_SELECTOR, 'div.thumbnail_thumb_wrap__RbcYO._wrapper > a')

for product_page in tqdm(product_pages, total=len(product_pages), desc='Processing Product Pages'):
    try:
        initial_tab_count = len(driver.window_handles)

        product_page.click()
        driver.implicitly_wait(5)

        new_window = driver.window_handles[-1]
        driver.switch_to.window(new_window)
        driver.implicitly_wait(5)
        
        driver.find_element(By.CSS_SELECTOR, 'div._3osy73V_eD._1Hc_ju_IXp > button').click()
        driver.implicitly_wait(5)

        html_content = driver.page_source

        soup = BeautifulSoup(html_content, 'html.parser')

        product_name = soup.select_one('h3._22kNQuEXmb._copyable').text

        print(f'Product Name: {product_name}')

        valid_product_name = ''.join(c for c in os.path.splitext(product_name)[0] if c.isalnum() or c in (' ', '_')).rstrip()
        valid_product_name = valid_product_name.replace(' ', '_')
        target_directory = os.path.abspath(f'naver/{valid_product_name}')

        os.makedirs(target_directory)
            
        image_tags = soup.select('.se-main-container img')

        for i, img in tqdm(enumerate(image_tags), total=len(image_tags), desc='Processing Images'):
            if 'data-src' in img.attrs and img['data-src'].strip():
                image_url = img['data-src']

                parsed_url = urlparse(image_url)
                filename, extension = os.path.splitext(os.path.basename(parsed_url.path))

                file_path = os.path.join(target_directory, f'image_{i}{extension}')

                response = requests.get(image_url)

                with open(file_path, 'wb') as f:
                    f.write(response.content)
        
        print(f'Images for product \'{valid_product_name}\' successfully processed.')
        
        driver.close()
        
        driver.switch_to.window(driver.window_handles[0])
    
    except Exception as e:
        print(f'Error processing images for product \'{valid_product_name}\': {str(e)}')
        pass 

driver.quit()

45


Processing Product Pages:   0%|          | 0/45 [00:00<?, ?it/s]

Product Name: [직영] 호주 강아지사료 1kg 캥거루 & 스위트포테이토, 펫죠 키블댓카운트


Processing Product Pages:   2%|▏         | 1/45 [00:19<14:25, 19.66s/it]

Product Name: [직영] 호주 강아지사료 4kg 3종 캥거루/살몬/치킨, 펫죠 키블댓카운트


...

Product Name: 프라임캣 골드 15kg 고양이사료 대용량 전연령 길냥이밥 길고양이사료 뉴트리나


Processing Product Pages:  33%|███▎      | 15/45 [04:00<08:21, 16.73s/it]

Product Name: 아스쿠 펠리쿠치나 반습식 소고기 1.2kg+150g


Processing Product Pages:  36%|███▌      | 16/45 [04:15<07:52, 16.29s/it]

Product Name: 버틀러 사료 더텐 소프트 인섹트 알러지케어 더 텐 1kg the ten soft + 샘플추가
